# Assignment 9 Example

*Last updated: 11/6/2019, Andrew Therriault*

This notebook gives an example of what a completed Week 9 assignment might look like. It builds a forecast of the 2020 Democratic Presidential Primary based on the polls collected by 538 (using the dataset as of 11/6/2019).

## Setting up the environment


#### Setting the working directory

In [1]:
import os
os.chdir('C:\\working\\')

#### Importing numpy and pandas for analysis and setting display options for pandas

In [2]:
import numpy as np

In [3]:
import pandas as pd
pd.options.display.max_colwidth = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

## Loading, transforming, and exploring data

#### Loading just the columns we actually care about

In [4]:
columns_to_keep = [
    'poll_id',
    'state',
    'pollster',
    'fte_grade',
    'sample_size',
    'population',
    'methodology',
    'start_date',
    'end_date',
    'tracking',
    'answer',
    'pct']

In [5]:
raw = pd.read_csv('president_primary_polls.csv', usecols=columns_to_keep, parse_dates=['start_date','end_date'])

#### Filling in some missing data

In [6]:
for i in ['fte_grade','methodology']:
    raw[i] = raw[i].fillna('Missing')

#### Filtering to just the candidates we care about
I'm limiting to the 9 candidates still in the race who have qualified for inclusion in the November 20th debate as of 11/5. (The only noteworthy exclusions based on this are Gabbard and Castro, but both are excluded because they don't show enough polling strength to meet the threshold, so that means they wouldn't affect a forecast much anyway.)

In [7]:
cands = [
    'Biden',
    'Harris',
    'Warren',
    'Sanders',
    'Booker',
    'Klobuchar',
    'Buttigieg',
    'Yang',
    'Steyer'
    ]

In [8]:
raw = raw[raw.answer.map(lambda x: x in cands)]

#### Limiting to states with at least three unique polls

In [9]:
raw['state'] = raw.state.fillna('National')
polls_by_state = raw.drop_duplicates(['state','poll_id']).groupby('state').poll_id.count()
states = polls_by_state[polls_by_state.map(lambda x: x >= 3)].index

In [10]:
raw = raw[raw.state.map(lambda x: x in states)]

In [11]:
raw.sample(5).T

,9744,12471,12613,2724,1918
poll_id,58104,57244,57185,59626,59757
state,National,National,Iowa,Iowa,National
pollster,Morning Consult,Morning Consult,Selzer & Co.,Selzer & Co.,Monmouth University
fte_grade,B/C,B/C,A+,A+,A+
sample_size,14830,699,455,602,434
population,lv,rv,lv,lv,rv
methodology,Online,Online,Live Phone,Live Phone,Live Phone
start_date,2019-05-13 00:00:00,2019-01-04 00:00:00,2018-12-10 00:00:00,2019-09-14 00:00:00,2019-09-23 00:00:00
end_date,2019-05-19 00:00:00,2019-01-06 00:00:00,2018-12-13 00:00:00,2019-09-18 00:00:00,2019-09-29 00:00:00
tracking,False,False,False,False,False


#### Looping over categorical variables to see value counts

In [12]:
for i in raw.dtypes[raw.dtypes == 'object'].index:
    print('\n{}:'.format(i))
    print(raw[i].value_counts()[:25])


state:
National          3377
New Hampshire      301
Iowa               266
South Carolina     251
California         214
Texas              152
Florida             96
Nevada              87
Wisconsin           77
Pennsylvania        55
Michigan            50
Massachusetts       40
North Carolina      40
Ohio                35
Arizona             34
Virginia            32
Georgia             26
Name: state, dtype: int64

pollster:
Harris Insights & Analytics                     1435
Morning Consult                                  432
Change Research                                  416
YouGov                                           403
Emerson College                                  252
Ipsos                                            163
Quinnipiac University                            162
Monmouth University                              141
Fox News/Beacon Research/Shaw & Co. Research      98
SSRS                                              97
Zogby Interactive/JZ Analytics    

#### Creating dataframe of polls for merging later (this is to allow for cleaning up polls which publish multiple sets of results)

Looking at poll_ids with multiple sets of results

In [13]:
polls = raw[['poll_id', 'state', 'pollster', 'fte_grade', 'sample_size', 'population', 'methodology',
                'start_date', 'end_date', 'tracking']].drop_duplicates()
result_count = polls.poll_id.value_counts()
multi_results = result_count[result_count > 1].index
polls[polls.poll_id.map(lambda x: x in multi_results)].sort_values(by='poll_id')[:20]

,poll_id,state,pollster,fte_grade,sample_size,population,methodology,start_date,end_date,tracking
12656,57064,National,SSRS,A/B,424,rv,Live Phone,2018-12-06,2018-12-09,False
12636,57064,National,SSRS,A/B,463,a,Live Phone,2018-12-06,2018-12-09,False
12700,57182,National,Harris Insights & Analytics,C+,449,rv,Online,2018-11-27,2018-11-28,False
12692,57182,National,Harris Insights & Analytics,C+,459,rv,Online,2018-11-27,2018-11-28,False
12408,57376,National,Harris Insights & Analytics,C+,488,rv,Online,2019-01-15,2019-01-16,False
12420,57376,National,Harris Insights & Analytics,C+,479,rv,Online,2019-01-15,2019-01-16,False
11834,57583,National,Harris Insights & Analytics,C+,346,rv,Online,2019-02-19,2019-02-20,False
11846,57583,National,Harris Insights & Analytics,C+,337,rv,Online,2019-02-19,2019-02-20,False
11307,57700,National,SSRS,A/B,411,rv,Live Phone,2019-03-14,2019-03-17,False
11290,57700,National,SSRS,A/B,456,a,Live Phone,2019-03-14,2019-03-17,False


Limiting to the set of results with the smallest sample size for each poll, which is the tightest screen (likely voters before registered voters, registered voters before all adults, etc.) 

In [14]:
polls = polls.sort_values(by='sample_size',ascending=True).drop_duplicates('poll_id')

Calculating age of polls and duration in the field

In [15]:
polls['age'] = (pd.Timestamp('now') - polls.end_date).dt.days
polls['duration'] = (polls.end_date - polls.start_date).dt.days + 1

In [16]:
polls.head()

,poll_id,state,pollster,fte_grade,sample_size,population,methodology,start_date,end_date,tracking,age,duration
9463,58460,South Carolina,Zogby Interactive/JZ Analytics,C,183,lv,Online,2019-05-23,2019-05-29,False,161,7
9388,58455,Arizona,Zogby Interactive/JZ Analytics,C,197,lv,Online,2019-05-23,2019-05-29,False,161,7
2263,62437,Michigan,Denno Research,Missing,217,lv,Live Phone,2019-09-21,2019-09-24,False,43,4
9433,58458,Ohio,Zogby Interactive/JZ Analytics,C,222,lv,Online,2019-05-23,2019-05-29,False,161,7
353,62520,Pennsylvania,Franklin & Marshall College,B/C,226,rv,Live Phone,2019-10-21,2019-10-27,False,10,7


#### Joining in results that match to those included poll_ids (with all the other fields matching)

In [17]:
results = polls.merge(raw, how='inner', on=['poll_id', 'state', 'pollster', 'fte_grade', 'sample_size', 'population', 
                                            'methodology', 'start_date', 'end_date', 'tracking'])
results.head()

,poll_id,state,pollster,fte_grade,sample_size,population,methodology,start_date,end_date,tracking,age,duration,answer,pct
0,58460,South Carolina,Zogby Interactive/JZ Analytics,C,183,lv,Online,2019-05-23,2019-05-29,False,161,7,Biden,36.2
1,58460,South Carolina,Zogby Interactive/JZ Analytics,C,183,lv,Online,2019-05-23,2019-05-29,False,161,7,Sanders,13.2
2,58460,South Carolina,Zogby Interactive/JZ Analytics,C,183,lv,Online,2019-05-23,2019-05-29,False,161,7,Warren,12.2
3,58460,South Carolina,Zogby Interactive/JZ Analytics,C,183,lv,Online,2019-05-23,2019-05-29,False,161,7,Buttigieg,6.9
4,58460,South Carolina,Zogby Interactive/JZ Analytics,C,183,lv,Online,2019-05-23,2019-05-29,False,161,7,Harris,3.8


#### Averaging across a handful of remaining duplicate entries
There aren't many of these, so I'm assuming they're additional variants not captured in the standard columns, and rather than deal with them individually I'm just averaging the results across them.

In [18]:
results = results.groupby(['poll_id', 'state', 'pollster', 'fte_grade', 'sample_size', 'population', 
                            'methodology', 'start_date', 'end_date', 'tracking', 'answer']
                            ).pct.mean().reset_index()
results.head()

,poll_id,state,pollster,fte_grade,sample_size,population,methodology,start_date,end_date,tracking,answer,pct
0,56853,National,Morning Consult,B/C,733,rv,Online,2018-11-07,2018-11-09,False,Biden,26.0
1,56853,National,Morning Consult,B/C,733,rv,Online,2018-11-07,2018-11-09,False,Booker,3.0
2,56853,National,Morning Consult,B/C,733,rv,Online,2018-11-07,2018-11-09,False,Harris,4.0
3,56853,National,Morning Consult,B/C,733,rv,Online,2018-11-07,2018-11-09,False,Klobuchar,1.0
4,56853,National,Morning Consult,B/C,733,rv,Online,2018-11-07,2018-11-09,False,Sanders,19.0


#### Pivoting long to wide (one column per candidate), filling in with zeros, and calculating undecideds

In [19]:
results_wide = results.pivot(index='poll_id', columns='answer', values='pct').fillna(0)
results_wide['TOTAL'] = results_wide.sum(axis=1)
results_wide['UNDECIDED'] = 100 - results_wide['TOTAL']
results_wide.head(10)

answer,Biden,Booker,Buttigieg,Harris,Klobuchar,Sanders,Steyer,Warren,Yang,TOTAL,UNDECIDED
poll_id,,,,,,,,,,,
56853,26.0,3.0,0.0,4.0,1.0,19.0,0.0,5.0,0.0,58.0,42.0
57064,33.0,6.0,0.0,4.0,3.0,14.0,0.0,4.0,0.0,64.0,36.0
57075,25.5,0.0,0.0,8.5,0.0,22.0,0.0,6.9,0.0,62.9,37.1
57144,25.0,3.0,0.0,3.0,1.0,15.0,0.0,3.0,0.0,50.0,50.0
57181,19.0,3.0,0.0,6.0,3.0,14.0,0.0,11.0,0.0,56.0,44.0
57182,28.0,4.0,0.0,3.0,0.0,21.0,0.0,5.0,0.0,61.0,39.0
57183,17.0,2.0,0.0,3.0,1.0,18.0,0.0,4.0,0.0,45.0,55.0
57184,30.0,5.0,0.0,5.0,0.0,20.0,0.0,5.0,0.0,65.0,35.0
57185,32.0,4.0,0.0,5.0,3.0,19.0,0.0,8.0,0.0,71.0,29.0


#### Making the final dataframe

In [20]:
data = polls.merge(results_wide, on='poll_id').set_index('poll_id').sort_index()
data.head(20)

,state,pollster,fte_grade,sample_size,population,methodology,start_date,end_date,tracking,age,duration,Biden,Booker,Buttigieg,Harris,Klobuchar,Sanders,Steyer,Warren,Yang,TOTAL,UNDECIDED
poll_id,,,,,,,,,,,,,,,,,,,,,,
56853,National,Morning Consult,B/C,733,rv,Online,2018-11-07,2018-11-09,False,362,3,26.0,3.0,0.0,4.00,1.0,19.0,0.0,5.0,0.0,58.00,42.00
57064,National,SSRS,A/B,424,rv,Live Phone,2018-12-06,2018-12-09,False,332,4,33.0,6.0,0.0,4.00,3.0,14.0,0.0,4.0,0.0,64.00,36.00
57075,National,Emerson College,A-,320,lv,IVR/Online,2018-12-06,2018-12-09,False,332,4,25.5,0.0,0.0,8.50,0.0,22.0,0.0,6.9,0.0,62.90,37.10
57144,National,Morning Consult,B/C,706,rv,Online,2018-12-14,2018-12-16,False,325,3,25.0,3.0,0.0,3.00,1.0,15.0,0.0,3.0,0.0,50.00,50.00
57181,Massachusetts,YouGov,B-,655,rv,Online,2018-11-07,2018-11-14,False,357,8,19.0,3.0,0.0,6.00,3.0,14.0,0.0,11.0,0.0,56.00,44.00
57182,National,Harris Insights & Analytics,C+,449,rv,Online,2018-11-27,2018-11-28,False,343,2,28.0,4.0,0.0,3.00,0.0,21.0,0.0,5.0,0.0,61.00,39.00
57183,National,McLaughlin & Associates,C/D,468,lv,Online,2018-12-10,2018-12-14,False,327,5,17.0,2.0,0.0,3.00,1.0,18.0,0.0,4.0,0.0,45.00,55.00
57184,National,Harris Insights & Analytics,C+,370,rv,Online,2018-11-05,2018-11-06,False,365,2,30.0,5.0,0.0,5.00,0.0,20.0,0.0,5.0,0.0,65.00,35.00
57185,Iowa,Selzer & Co.,A+,455,lv,Live Phone,2018-12-10,2018-12-13,False,328,4,32.0,4.0,0.0,5.00,3.0,19.0,0.0,8.0,0.0,71.00,29.00


## Setting up the forecast function

This function combines poll results using the following fixed rules and options:
* Fixed rules:
  * Each poll result is given a weight based on a variety of different characteristics (see "options" below), and the weighted average of these is used as the forecast. (This means that we're estimating where the race stands now, not where it's going to be in the future - there's no imputation of a future state, e.g. by forecasting a trend line.)
  * Each poll characteristic is given a weight value, and these are all multiplied together to calculate the overall weight per poll.
  * "Undecided" is treated as a distinct response choice, 
* Options: 
  * Which state the calculation is for ("National" is treated as a separate state here)
  * Which candidates are included
  * Max number of poll results included per pollster (keeps only most recent)
  * Max age of poll results included
  * A set of pre-defined weights based on values of:
    * fte_grade (pollster rating by 538)
    * population
    * methodology
    * tracking
  * A set of weighting functions to apply to values of:
    * sample_size
    * duration
    * age
    
Some things *not* included in here that could be:
 * Adjusting to down-weight additional polls from the same pollster
 * Giving lower weights to older results from the same pollster (other than because of increased age)
 * Removing results from tracking polls with overlapping timeframes (but 538 has done some of this already, and the weight given to the "tracking" flag can partially adjust for this)

#### Setting pre-defined weights for values of each characteristic (as dicts)
Adding a "Missing" option for each (which will get filled in during processing) except for Tracking, where missing values are treated as False

In [21]:
w_fte_grade = {
    'A+' : 4.3,
    'A' : 4.0,
    'A-' : 3.7,
    'A/B' : 3.5,
    'B+' : 3.3,
    'B' : 3.0,
    'B-' : 2.7,
    'B/C' : 2.5,
    'C+' : 2.3,
    'C' : 2.0,
    'C/D' : 1.5,
    'D-' : 0.7,
    'Missing' : 1.0 
}

In [22]:
w_population = {
    'lv' : 2.0,
    'rv' : 1.5,
    'v' : 1.0,
    'a' : 1.0,
    'Missing' : 1.0
}

In [23]:
w_methodology = {
    'Live Phone' : 3.0,
    'Live Phone/Text' : 3.0,
    'Live Phone/Online' : 3.0,
    'Online' : 2.0,
    'IVR/Text': 2.0,
    'Online/IVR' : 2.0,
    'IVR/Online' : 2.0,
    'Automated Phone': 1.0,
    'Missing' : 1.0
}

In [24]:
w_tracking = {
    True: 0.5,
    False: 1.0,
}

#### Setting weighting functions for sample size, poll age, and duration

In [25]:
def f_sample_size(x):
    z = np.amin([x,2000]) #topcoding at 2000 to avoid over-weighting giant online panels
    return (z**0.5)/(1000**0.5)

#showing how it works in practice
for i in [0,100,500,1000,2000,10000]:
    print('{}: {}'.format(i,f_sample_size(i)))

0: 0.0
100: 0.31622776601683794
500: 0.7071067811865476
1000: 1.0
2000: 1.4142135623730951
10000: 1.4142135623730951


In [26]:
def f_age(x):
    return 3 / (x+9)**0.5

#showing how it works in practice
for i in [0,3,7,30,90,365]:
    print('{}: {}'.format(i,f_age(i)))

0: 1.0
3: 0.8660254037844387
7: 0.75
30: 0.48038446141526137
90: 0.30151134457776363
365: 0.15512630699850574


In [27]:
def f_duration(x):
    if (x > 0) & (x < 4):
        return x**0.5
    elif x >= 4:
        return 2.0

#showing how it works in practice
for i in [0,1,2,3,4,5]:
    print('{}: {}'.format(i,f_duration(i)))

0: None
1: 1.0
2: 1.4142135623730951
3: 1.7320508075688772
4: 2.0
5: 2.0


#### Creating the forecast function

In [28]:
def forecast(x, state, candidates, max_results, max_age,
                p_fte_grade, p_population, p_methodology, p_tracking,
                p_sample_size, p_age, p_duration):
    
    #limiting to the state being forecast and the max number of polls per pollster and max poll age
    x = x[x.state==state].copy()
    x['recency'] = x.sort_values(by='age', ascending=True).groupby('pollster').cumcount()
    x = x[x.recency < max_results]
    x = x[x.age <= max_age]
    
    #applying weight mappings
    x['w1'] = x.fte_grade.map(p_fte_grade)
    x['w2'] = x.population.map(p_population)
    x['w3'] = x.methodology.map(p_methodology)
    x['w4'] = x.tracking.map(p_tracking)
    x['w5'] = x.sample_size.map(p_sample_size)
    x['w6'] = x.age.map(p_age)
    x['w7'] = x.duration.map(p_duration)
    
    #calculating overall weights and adjusting to mean=1
    x['w'] = x.w1 * x.w2 * x.w3 * x.w4 * x.w5 * x.w6 * x.w7
    x['w'] = x.w / x.w.mean()
    
    #calculating forecasts
    forecast = dict()
    for i in (candidates + ['UNDECIDED']):
        forecast[i] = np.round((x[i]*x.w).sum() / x.w.sum(),1)
    
    #showing metadata
    print('\n\nFORECAST: {}'.format(state))
    print('\nNumber of polls included: {}'.format(len(x)))
    top5 = x.sort_values(by='w', ascending=False).head(5)[
            ['pollster', 'fte_grade', 'sample_size', 'population', 'methodology', 'end_date',
             'tracking', 'age', 'duration', 'w']]
    print('\nTop 5 highest-weighted polls:')
    print(top5)
        
    #showing forecast
    print('\nForecast:')
    print(pd.Series(forecast).sort_values(ascending=False))

## Generating forecasts

#### National forecast

In [29]:
forecast(data, 'National', candidates=cands, max_results=3, max_age = 90,       
            p_fte_grade=w_fte_grade, p_population=w_population, p_methodology=w_methodology, p_tracking=w_tracking,
            p_sample_size=f_sample_size, p_age=f_age, p_duration=f_duration)



FORECAST: National

Number of polls included: 55

Top 5 highest-weighted polls:
                                             pollster fte_grade  sample_size  \
poll_id                                                                        
62557                                 Morning Consult       B/C        16071   
62536    Fox News/Beacon Research/Shaw & Co. Research        A-          471   
62537                    NBC News/Wall Street Journal       A/B          414   
62563                             Monmouth University        A+          345   
62503                                 Morning Consult       B/C        15431   

        population methodology   end_date  tracking  age  duration         w  
poll_id                                                                       
62557           lv      Online 2019-11-03     False    3         7  2.544276  
62536           lv  Live Phone 2019-10-30     False    7         4  2.373795  
62537           lv  Live Phone 2019-10-30

#### State-by-state forecasts

In [30]:
for i in sorted(data.state.unique()):
    if i != 'National':
        forecast(data, state=i, candidates=cands, max_results=3, max_age = 365,       
                    p_fte_grade=w_fte_grade, p_population=w_population, p_methodology=w_methodology, p_tracking=w_tracking,
                    p_sample_size=f_sample_size, p_age=f_age, p_duration=f_duration)



FORECAST: Arizona

Number of polls included: 4

Top 5 highest-weighted polls:
                                pollster fte_grade  sample_size population  \
poll_id                                                                      
62508                    Emerson College        A-          339         lv   
59598    Bendixen & Amandi International       B/C          250         rv   
59778                    Change Research         C          396         lv   
58455     Zogby Interactive/JZ Analytics         C          197         lv   

        methodology   end_date  tracking  age  duration         w  
poll_id                                                            
62508    IVR/Online 2019-10-28     False    9         4  2.307815  
59598    Live Phone 2019-09-12     False   55         4  0.798929  
59778        Online 2019-09-28     False   39         2  0.583818  
58455        Online 2019-05-29     False  161         7  0.309438  

Forecast:
Biden        26.8
Warren       2


Top 5 highest-weighted polls:
                    pollster fte_grade  sample_size population  \
poll_id                                                          
62556          Mellman Group       B/C          600         lv   
62539        Emerson College        A-          451         lv   
59640     Suffolk University        A-          500         lv   
59739                   SSRS       A/B          324         lv   
58427    Monmouth University        A+          370         lv   

             methodology   end_date  tracking  age  duration         w  
poll_id                                                                 
62556    Live Phone/Text 2019-11-02     False    4         6  2.318150  
62539         IVR/Online 2019-11-03     False    3         4  2.063981  
59640         Live Phone 2019-09-23     False   44         5  1.551124  
59739         Live Phone 2019-09-26     False   41         5  1.216055  
58427         Live Phone 2019-06-11     False  148         6  0.9009


Top 5 highest-weighted polls:
                      pollster fte_grade  sample_size population methodology  \
poll_id                                                                        
59725    Research America Inc.       B/C          882         rv  Live Phone   
58509       Hampton University       B/C         1126         rv      Online   
58000          Change Research         C          551         lv      Online   

          end_date  tracking  age  duration         w  
poll_id                                                
59725   2019-09-15     False   52        13  1.691659  
58509   2019-06-06     False  153         9  0.781923  
58000   2019-04-30     False  190         5  0.526417  

Forecast:
UNDECIDED    30.0
Biden        29.5
Sanders      13.0
Warren       10.2
Buttigieg     7.2
Harris        5.5
Yang          1.6
Booker        1.6
Klobuchar     1.3
Steyer        0.0
dtype: float64


FORECAST: Wisconsin

Number of polls included: 9

Top 5 highest-weighted polls:
